## Imports


In [2]:
from universal import UniversalDS, ChrData
from topologicalFeatures import *
from GOEAa import *
from vizFileMaker import VisData

## Create dataset object for Hi-C data

In [19]:
import sys
import os
from pathlib import Path
from folderLocation import getLocalFolder

SCRIPT_DIR = str(getLocalFolder())
#LZPdir = Path(SCRIPT_DIR).parents[1] #goues up 2 directories
#fn = str(os.path.join(LZPdir, "Processed_Data", "Blood", "data-pvalue-5-fin.json"))
# commented out code works for .py files
fn = SCRIPT_DIR
LZPdir = fn[:fn.index("LZP")] + "LZP/"

fn = fn[:fn.index("LZP")] + "LZP/Processed_Data/Blood/data-pvalue-5-fin.json"
#found the path to the demo file in universal format

U = UniversalDS(fn)

## Create chromosome object with filtered links
Only links that are in at least minLinkTissueCount tissues and have all of these tissues - tissueMask (either list, str or int) - are added.
chData is an object that stores the filtered sub-graph

In [13]:
chData = ChrData(U, ch="chr9", minLinkTissueCount=2, tissueMask=["Mac1", "Mac2"])
links = chData.links

## Getting summary for current chromosome


In [14]:
summary = chData.summarize()
print(summary)

{'Chromosome': 'chr9', 'Nodes count': 2975, 'Link count': 4036, 'Giant component nodes': 295, 'Giant component links': 364, 'Link percentage in Giant component': 0.09018830525272548}


# Start calculating topological features
## Calculate triangles

In [15]:
C = Cliques(chData, minC3TissueCount=3, tissueMask=0) 
#all filtered triangles will have at least 3 tissues of any type
triangles = C.allCliques
if (len(triangles)>0): 
    print("one of the {} triangles found in this chr: {} ".format(len(triangles), str(triangles[0])))
    #now writing this triangle in loci and tissue list form to demonstrate how translation can be done, if necessary
    tr = triangles[0]
    t = [chData.segmentIndToMidpoint[tr[i]] for i in [0,1,2]]+[U.getTissueNameList(tr[3])]
    print(t)


one of the 395 triangles found in this chr: (9034, 9038, 9062, 105727) 
[139694137, 139709892, 139934163, ['aCD4', 'EP', 'Ery', 'FoeT', 'Mac0', 'Mac1', 'Mac2', 'MK', 'nB', 'nCD4', 'nCD8', 'tCD4', 'tCD8']]


### Visualize these triangles

In [16]:
bb = VisData(C, min_samples=20, eps=30000, DSName="demoResults/demoFileForVisualization.json")
#created VisData instance that will make a file for visualization for all chromosomes with the same filters as for C object calculated before
#file demoFileForVisualization.sjon is created and can be used in cisualization

## Calculate bases

### There are 2 types of bases - AND-bases and OR-bases
#### Calculating both for all tissue types

In [17]:
Bor = Bases(chData)
Bor.setBasesOr()

Band = Bases(chData)
Band.setBasesAnd()

#now calculate those bases that are in at least 4 or 5 triangles
BorBases4 = Bor.getDegreeNBases(4)
#show some bases4 in tissue type Mac1
print(BorBases4["Mac1"][:4])
#for each base ling endpoint indeces and the degree are shown

BandBases4 = Band.getDegreeNBases(5)
print(BandBases4["Mac1"][:4])

#get nodes that are in a base
stru = Bor.getSegmentStructureInds(N=4, filename="demoResults/baseSegmentStructureDemo.json")

[[1970, 2038, 4], [1406, 1436, 22], [1067, 1101, 29], [1890, 1970, 18]]
[[1406, 1436, 22], [1067, 1101, 29], [1890, 1970, 18], [5657, 5761, 5]]


# Do gene ontology enrichment analysis

In [20]:
Biolog = GOEATool("ontologyData/go-basic.obo", "ontologyData/hg19_genes_w-go.txt")
# Give necessary data files from public databases

ontologyData/go-basic.obo: fmt(1.2) rel(2022-10-07) 46,824 Terms


In [21]:
countFound = Biolog.GOEA(stuObject=Bor, popObject=C)
print(countFound)

       47 READ: stu-tmp.tsv
      126 READ: pop-tmp.tsv
**WARNING: GO:0000060 NOT FOUND IN DAG
**WARNING: GO:0000185 NOT FOUND IN DAG
**WARNING: GO:0000186 NOT FOUND IN DAG
**WARNING: GO:0000187 NOT FOUND IN DAG
**WARNING: GO:0000188 NOT FOUND IN DAG
**WARNING: GO:0001076 NOT FOUND IN DAG
**WARNING: GO:0001190 NOT FOUND IN DAG
**WARNING: GO:0001300 NOT FOUND IN DAG
**WARNING: GO:0003256 NOT FOUND IN DAG
**WARNING: GO:0004221 NOT FOUND IN DAG
**WARNING: GO:0004716 NOT FOUND IN DAG
**WARNING: GO:0004871 NOT FOUND IN DAG
**WARNING: GO:0005057 NOT FOUND IN DAG
**WARNING: GO:0005070 NOT FOUND IN DAG
**WARNING: GO:0005072 NOT FOUND IN DAG
**WARNING: GO:0005671 NOT FOUND IN DAG
**WARNING: GO:0006184 NOT FOUND IN DAG
**WARNING: GO:0006200 NOT FOUND IN DAG
**WARNING: GO:0006927 NOT FOUND IN DAG
**WARNING: GO:0006928 NOT FOUND IN DAG
**WARNING: GO:0006987 NOT FOUND IN DAG
**WARNING: GO:0007016 NOT FOUND IN DAG
**WARNING: GO:0007257 NOT FOUND IN DAG
**WARNING: GO:0008144 NOT FOUND IN DAG
**WARNIN

In [ ]:
rez = []
for ch in ["chr6", "chr19"]:
    chData = ChrData(U, ch=ch)
    C = Cliques(owner=chData, minC3TissueCount=2)
    count = Biolog.GOEA(stuObject=C, popObject=chData) #how many found terms with pv<0.05
    if count>0:
        rez.extend([str(row) for row in Biolog.curResultSig])
print (rez)
    

In [23]:
import csv
with open('demoResults/demoGOresult.tsv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    for row in rez:
        writer.writerow(row.split('\t'))

In [24]:
print(rez[0])



GO:0065004	BP	e	protein-DNA complex assembly  	62/651	62/953	2.97e-11	6	62	1.69e-07	ASF1A, BRD2, CENPQ, HEY2, HIST1H1A, HIST1H1B, HIST1H1C, HIST1H1D, HIST1H1E, HIST1H1T, HIST1H2AA, HIST1H2AB, HIST1H2AC, HIST1H2AD, HIST1H2AE, HIST1H2AG, HIST1H2AH, HIST1H2AI, HIST1H2AJ, HIST1H2AK, HIST1H2AL, HIST1H2AM, HIST1H2BA, HIST1H2BB, HIST1H2BC, HIST1H2BD, HIST1H2BE, HIST1H2BF, HIST1H2BG, HIST1H2BH, HIST1H2BI, HIST1H2BJ, HIST1H2BK, HIST1H2BL, HIST1H2BM, HIST1H2BN, HIST1H2BO, HIST1H3A, HIST1H3B, HIST1H3C, HIST1H3D, HIST1H3E, HIST1H3F, HIST1H3G, HIST1H3H, HIST1H3I, HIST1H3J, HIST1H4A, HIST1H4B, HIST1H4C, HIST1H4D, HIST1H4E, HIST1H4F, HIST1H4G, HIST1H4H, HIST1H4I, HIST1H4J, HIST1H4K, HIST1H4L, SHPRH, TSPYL1, TSPYL4
